In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from streamlit import dataframe as sdf
import streamlit as st

def create_retail_dataset(seed=42):
    """
    Creates an artificial retail dataset with specified parameters.
    
    Returns:
        pandas.DataFrame: The generated dataset
    """
    # Set random seed for reproducibility
    np.random.seed(seed)
    random.seed(seed)
    
    # Constants
    NUM_CUSTOMERS = 10000
    START_DATE = datetime(2024, 1, 1)
    END_DATE = datetime(2025, 1, 3)
    CHANNEL_SWITCH_DATE = datetime(2024, 6, 1)
    MEMBERSHIP_CHANGE_START = datetime(2024, 6, 1)
    MEMBERSHIP_CHANGE_END = datetime(2024, 7, 1)
    MIN_AMOUNT = 10
    MAX_AMOUNT = 150
    INITIAL_MEMBER_RATIO = 0.25
    ADDITIONAL_CONVERSION_RATIO = 0.15
    
    # Generate customer data
    customer_ids = [f"CUST_{i:06d}" for i in range(1, NUM_CUSTOMERS + 1)]
    genders = np.random.choice(['MALE', 'FEMALE'], size=NUM_CUSTOMERS)
    
    # Initial membership status - 25% are MEMBERs
    initial_memberships = np.random.choice(
        ['MEMBER', 'NON_MEMBER'], 
        size=NUM_CUSTOMERS, 
        p=[INITIAL_MEMBER_RATIO, 1-INITIAL_MEMBER_RATIO]
    )
    
    # Determine which non-members will convert to members during the conversion period
    # First identify all non-members
    non_member_indices = [i for i, status in enumerate(initial_memberships) if status == 'NON_MEMBER']
    # Calculate how many should convert (15% of non-members)
    num_to_convert = int(len(non_member_indices) * ADDITIONAL_CONVERSION_RATIO / (1 - INITIAL_MEMBER_RATIO))
    # Randomly select which ones convert
    convert_indices = np.random.choice(non_member_indices, size=num_to_convert, replace=False)
    
    # Create customer lookup dictionary with conversion date for those who will convert
    customers = {}
    for i in range(NUM_CUSTOMERS):
        conversion_date = None
        if i in convert_indices:
            # Randomly select a date in the conversion period
            days_in_period = (MEMBERSHIP_CHANGE_END - MEMBERSHIP_CHANGE_START).days
            random_days = random.randint(0, days_in_period - 1)
            conversion_date = MEMBERSHIP_CHANGE_START + timedelta(days=random_days)
            
        customers[customer_ids[i]] = {
            'gender': genders[i],
            'initial_membership': initial_memberships[i],
            'conversion_date': conversion_date
        }
    
    # Calculate weeks in the date range
    current_date = START_DATE
    weeks = []
    while current_date <= END_DATE:
        weeks.append(current_date)
        current_date += timedelta(days=7)
    
    # Generate transactions
    data = []
    
    for week_start in weeks:
        week_end = week_start + timedelta(days=6)
        for customer_id, attributes in customers.items():
            # 1-3 transactions per week per customer
            num_transactions = random.randint(1, 3)
            
            for _ in range(num_transactions):
                # Generate transaction date within the week
                days_offset = random.randint(0, 6)
                transaction_date = week_start + timedelta(days=days_offset)
                
                # Determine membership status for this transaction date
                if attributes['initial_membership'] == 'MEMBER':
                    membership_status = 'MEMBER'
                elif attributes['conversion_date'] and transaction_date >= attributes['conversion_date']:
                    membership_status = 'MEMBER'
                else:
                    membership_status = 'NON_MEMBER'
                
                # Determine transaction channel based on date
                if transaction_date < CHANNEL_SWITCH_DATE:
                    channel_prob = 0.75
                    transaction_channel = 'IN_SHOP' if random.random() < channel_prob else 'ONLINE'
                else:
                    channel_prob = 0.25
                    transaction_channel = 'IN_SHOP' if random.random() < channel_prob else 'ONLINE'
                
                # Base transaction amount
                base_amount = random.uniform(MIN_AMOUNT, MAX_AMOUNT)
                
                # Apply gender modifier
                gender_modifier = random.uniform(1.1, 1.3) if attributes['gender'] == 'MALE' else 1.0
                
                # Apply membership modifier
                membership_modifier = random.uniform(1.1, 1.3) if membership_status == 'MEMBER' else 1.0
                
                # Final transaction amount
                transaction_amount = round(base_amount * gender_modifier * membership_modifier, 2)
                
                # Add transaction to data
                data.append({
                    'CUSTOMER_ID': customer_id,
                    'DATE': transaction_date.strftime('%Y-%m-%d'),
                    'TRANSACTION_AMOUNT': transaction_amount,
                    'TRANSACTION_CHANNEL': transaction_channel,
                    'GENDER': attributes['gender'],
                    'MEMBERSHIP_STATUS': membership_status
                })
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Balance monthly transaction amounts
    df['MONTH'] = pd.to_datetime(df['DATE']).dt.to_period('M')
    monthly_totals = df.groupby('MONTH')['TRANSACTION_AMOUNT'].sum()
    target_monthly_average = monthly_totals.mean()
    
    for month in monthly_totals.index:
        month_mask = df['MONTH'] == month
        current_total = monthly_totals[month]
        adjustment_factor = target_monthly_average / current_total
        df.loc[month_mask, 'TRANSACTION_AMOUNT'] = df.loc[month_mask, 'TRANSACTION_AMOUNT'] * adjustment_factor
    
    # Round transaction amounts to 2 decimal places
    df['TRANSACTION_AMOUNT'] = df['TRANSACTION_AMOUNT'].round(2)
    
    # Drop temporary column and sort by date
    df = df.drop(columns=['MONTH']).sort_values('DATE')

    df['DATE'] = pd.to_datetime(df['DATE'])
    df['DATE'] = df['DATE'].dt.date
    df = df.reset_index(drop=True)
    
    return df

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Generate and store customers and transactions 
transactions = create_retail_dataset()

# Create dataset with transactions
transactions_df = session.write_pandas(transactions.drop(['GENDER','MEMBERSHIP_STATUS'], axis=1), table_name='_TRANSACTIONS', database='SIMPLE_MLOPS_DEMO', schema='_DATA_GENERATION', auto_create_table=True, overwrite=True)


# Create dataset with customer history 
# Get the status for each customer on 2024-01-01
first_row_per_customer = transactions.drop(['TRANSACTION_AMOUNT','TRANSACTION_CHANNEL'], axis=1).sort_values(by=['CUSTOMER_ID', 'DATE']).drop_duplicates(subset=['CUSTOMER_ID'], keep='first')
first_row_per_customer['DATE'] = '2024-01-01'

# Get the customer history (changes for membership status) for every customer
customers_df = transactions.sort_values(by=['CUSTOMER_ID', 'DATE'])
customers_df['STATUS_CHANGE'] = customers_df['MEMBERSHIP_STATUS'] != customers_df.groupby('CUSTOMER_ID')['MEMBERSHIP_STATUS'].shift()
customers_df = customers_df[customers_df['STATUS_CHANGE']].drop(['TRANSACTION_AMOUNT', 'TRANSACTION_CHANNEL', 'STATUS_CHANGE'], axis=1)

# Combine both
customers_df = pd.concat([customers_df, first_row_per_customer])
customers_df['DATE'] = pd.to_datetime(customers_df['DATE'])
customers_df['DATE'] = customers_df['DATE'].dt.date
customers_df = customers_df.sort_values(by=['CUSTOMER_ID', 'DATE'])
customers_df = customers_df.drop_duplicates().reset_index(drop=True)

customers_df = session.write_pandas(customers_df, table_name='_CUSTOMERS', database='SIMPLE_MLOPS_DEMO', schema='_DATA_GENERATION', auto_create_table=True, overwrite=True)

In [ ]:
customers_df.show()

In [ ]:
transactions_df.show()

In [ ]:
# Create a DataFrame from the records
revenue_df = pd.DataFrame(records, columns=['DATE', 'REVENUE'])
revenue_df = session.create_dataframe(revenue_df)
revenue_in_shop = revenue_df.filter(col('DATE') < lit('2024-06-01'))
revenue_online = revenue_df.filter(col('DATE') >= lit('2024-06-01'))

revenue_in_shop.join_table_function('SIMPLE_MLOPS_DEMO._DATA_GENERATION.GENERATE_TRANSACTIONS', col('REVENUE'), lit([0.75,0.25])).drop('REVENUE').write.save_as_table(table_name='SIMPLE_MLOPS_DEMO._DATA_GENERATION._TRANSACTIONS', mode='overwrite')
revenue_online.join_table_function('SIMPLE_MLOPS_DEMO._DATA_GENERATION.GENERATE_TRANSACTIONS',col('REVENUE'), lit([0.25,0.75])).drop('REVENUE').write.save_as_table(table_name='SIMPLE_MLOPS_DEMO._DATA_GENERATION._TRANSACTIONS', mode='append')
session.table('SIMPLE_MLOPS_DEMO._DATA_GENERATION._TRANSACTIONS').select('CUSTOMER_ID').distinct().write.save_as_table('SIMPLE_MLOPS_DEMO._DATA_GENERATION._CUSTOMERS')
return "Demo Environment is setup."

In [ ]:
USE ROLE ACCOUNTADMIN;
-- Enable logs, metrics and traces
ALTER DATABASE SIMPLE_MLOPS_DEMO SET LOG_LEVEL = INFO;
ALTER DATABASE SIMPLE_MLOPS_DEMO SET METRIC_LEVEL = ALL;
ALTER DATABASE SIMPLE_MLOPS_DEMO SET TRACE_LEVEL = ALWAYS;

-- Create warehouses
CREATE WAREHOUSE IF NOT EXISTS COMPUTE_WH WITH WAREHOUSE_SIZE='X-SMALL';
CREATE WAREHOUSE IF NOT EXISTS FEATURE_STORE_WH WITH WAREHOUSE_SIZE='MEDIUM';

-- Create schema for setup
CREATE OR REPLACE SCHEMA SIMPLE_MLOPS_DEMO._DATA_GENERATION;

CREATE OR REPLACE FUNCTION SIMPLE_MLOPS_DEMO._DATA_GENERATION.GENERATE_TRANSACTIONS (REVENUE FLOAT, CHANNEL ARRAY)
  returns TABLE (CUSTOMER_ID STRING, TRANSACTION_AMOUNT FLOAT, TRANSACTION_CHANNEL STRING)
  language python
  runtime_version = '3.11'
  packages=('numpy','scipy')
  handler = 'GenerateTransactions'
as
$$
from scipy.stats import truncnorm
import numpy as np

class GenerateTransactions:
    # Draw a number from a normal distribution with defined mean, std_dev, lower and upper bounds
    def get_norm_value(self, min, max, mean, std_dev):
        # Calculate the a and b parameters for truncnorm
        min = (min - mean) / std_dev
        max = (max - mean) / std_dev
        
        # Generate the truncated normal distribution
        truncated_data = truncnorm.rvs(min, max, loc=mean, scale=std_dev, size=1)[0]
        return truncated_data
        
    def process(self, revenue, in_shop_online):
        customer_id = 0
        while revenue > 0:
            # customer id
            if (customer_id >= 0) and (customer_id < 100):
                transaction_amount = np.round(self.get_norm_value(5, 25, 20, 5),2)
            elif (customer_id >= 100) and (customer_id < 200):
                transaction_amount = np.round(self.get_norm_value(25, 50, 40, 5),2)
            elif (customer_id >= 200) and (customer_id < 300):
                transaction_amount = np.round(self.get_norm_value(50, 100, 80, 10),2)
            else:
                transaction_amount = np.round(self.get_norm_value(100, 150, 120, 10),2)
            transaction_channel = np.random.choice(['IN_SHOP','ONLINE'],p=in_shop_online)
            revenue = revenue - transaction_amount
            if customer_id == 1000:
                customer_id = 0
            customer_id = customer_id +1
            customer_id_str = f"CUST_{customer_id:06d}"
            yield (customer_id_str, transaction_amount, transaction_channel) 
$$
;

-- Setup Procedure
CREATE OR REPLACE PROCEDURE SIMPLE_MLOPS_DEMO._DATA_GENERATION.DATA_GENERATION()
  RETURNS STRING
  LANGUAGE PYTHON
  RUNTIME_VERSION = '3.11'
  PACKAGES = ('snowflake-snowpark-python','pandas','numpy')
  HANDLER = 'run'
  AS
$$
def run(session):
    from snowflake.snowpark.functions import lit, col
    import pandas as pd
    import numpy as np

    # Define the date range
    start_date = '2024-01-01'
    end_date = '2025-01-31'
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Define base revenue (this is a baseline you can adjust)
    base_revenue = 50000
    
    # Prepare a list to store computed records
    records = []
    
    for date in dates:
        # Determine the day-of-week: Monday=0, ... , Sunday=6
        weekday = date.weekday()
        
        # Weekday effect: Saturdays have the highest revenue,
        # Sundays are lower, and the rest are normal.
        if weekday == 5:       # Saturday
            weekday_factor = 1.5
        elif weekday == 6:     # Sunday
            weekday_factor = 0.9
        else:
            weekday_factor = 1.0
        
        # Month effect: June, July, August, and December get a boost.
        if date.month in [6, 7, 8, 12]:
            month_factor = 1.15
        else:
            month_factor = 1.0
        
        # Add random noise (mean 1, small standard deviation) to simulate natural fluctuations
        noise_factor = np.random.normal(loc=1, scale=0.05)
        
        # Compute the final revenue
        revenue = base_revenue * weekday_factor * month_factor * noise_factor
        revenue = max(0, revenue)  # Ensure no negative revenue
        
        # Append the result as a tuple (DATE, REVENUE)
        records.append((date, revenue))
    
    # Create a DataFrame from the records
    revenue_df = pd.DataFrame(records, columns=['DATE', 'REVENUE'])
    revenue_df = session.create_dataframe(revenue_df)
    revenue_in_shop = revenue_df.filter(col('DATE') < lit('2024-06-01'))
    revenue_online = revenue_df.filter(col('DATE') >= lit('2024-06-01'))
    
    revenue_in_shop.join_table_function('SIMPLE_MLOPS_DEMO._DATA_GENERATION.GENERATE_TRANSACTIONS', col('REVENUE'), lit([0.75,0.25])).drop('REVENUE').write.save_as_table(table_name='SIMPLE_MLOPS_DEMO._DATA_GENERATION._TRANSACTIONS', mode='overwrite')
    revenue_online.join_table_function('SIMPLE_MLOPS_DEMO._DATA_GENERATION.GENERATE_TRANSACTIONS',col('REVENUE'), lit([0.25,0.75])).drop('REVENUE').write.save_as_table(table_name='SIMPLE_MLOPS_DEMO._DATA_GENERATION._TRANSACTIONS', mode='append')
    session.table('SIMPLE_MLOPS_DEMO._DATA_GENERATION._TRANSACTIONS').select('CUSTOMER_ID').distinct().write.save_as_table('SIMPLE_MLOPS_DEMO._DATA_GENERATION._CUSTOMERS')
    return "Demo Environment is setup."
$$
;

In [ ]:
CALL SIMPLE_MLOPS_DEMO._DATA_GENERATION.DATA_GENERATION();

# Use Case: Predicting Future Customer Revenue Using Historical Transaction Data 📊

# 1 - Setup Demo 🛠️
* Import required libraries
* Create a Snowpark session

| Library    | Use |
| -------- | ------- |
| `snowflake.snowpark` | Main Python Developer Framework for Snowflake including the DataFrame-API     |
| `snowflake.ml`    | Snowflake ML specific functions including Feature Store & Model Registry APIs    |
| `snowflake.cortex`    | Snowflake APIs to access Cortex Services (e.g. LLMs)    |
| `notebook_extras`  | Convenience Functions for Snowflake Notebooks. More details [here](google.com).    |
| `demo_extras`  | Demo-specific functions (Data Generation, Use Case flow, etc.)     |

In [ ]:
# Helper functions for this demo
from demo_extras.flow import Demoflow
from demo_extras.model_trainer import ModelTrainer
from notebook_extras.cortex import CortexPilot
from notebook_extras.model_registry import ModelRegistryHelper
from notebook_extras.lineage import LineageHelper
from notebook_extras.misc import get_snowsight_url


# Import python packages
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
from streamlit import dataframe as sdf
import pandas as pd
import numpy as np
import shap
import warnings
import logging
from opentelemetry import trace
from snowflake import telemetry
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Import Snowflake packages
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import lit, col, sproc
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.metrics import mean_absolute_percentage_error
from snowflake.ml.registry import Registry
from snowflake.ml.monitoring.entities.model_monitor_config import ModelMonitorSourceConfig, ModelMonitorConfig
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)
from snowflake.core import Root, CreateMode

demo_flow = Demoflow()
demo_flow.setup()

session = get_active_session()

# 2 - Data Exploration & Visualization

* `session.table()` creates a reference to a table
* `count()`, `order_by()`, `describe()` are dataframe operations
* `describe()` gives us insights into the transaction amounts (e.g. min, average, max, count).

We can see that we have roughly 50K transactions across 350 customers.

In [ ]:
customers_df = session.table('SIMPLE_MLOPS_DEMO.RETAIL_DATA.CUSTOMERS')
transactions_df = session.table('SIMPLE_MLOPS_DEMO.RETAIL_DATA.TRANSACTIONS')

print(f'Number of customers: {customers_df.select("CUSTOMER_ID").distinct().count()}')
print('Customer Data:')
customers_df.order_by(col('CUSTOMER_ID'),col('DATE')).show(n=5)

print('Quick Variable Analysis:')
customers_df.describe().order_by('SUMMARY').show()

print(f'Number of transactions: {transactions_df.count()}')

print('Transactions Data:')
transactions_df.order_by(col('CUSTOMER_ID'),col('DATE')).show(n=5)

print('Quick Variable Analysis:')
transactions_df.describe().order_by('SUMMARY').show()

### Plotting Data
You can use libraries such as plotly or matplotlib to visualize your data. However, instead of coding the plots manually, we'll leverage GenAI models hosted natively in Snowflake to automatically generate the visualizations.

* This notebook comes with your own personal 🤖 **CortexPilot** powered by Cortex LLMs   
    * `ui_plotting()` -> UI-driven plotting with GenAI
    * `f_cortex_helper_visualize_query()` -> function that receives a Snowpark or Pandas Dataframe and a prompt (in case you already know the dataframe and query)

Both functions utilize Snowflake's [complete()](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/cortex/snowflake.cortex.complete) function to access LLMs natively hosted in Snowflake.

Try asking the following questions:  
* ***What was the overall revenue per channel and month? Use a stacked bar plot and use YY-Monthname for the x-axis. Make sure the x-axis is ordered.***
* ***What was the total transaction amount per channel? Use a pie chart.***

In [ ]:
from snowflake.cortex import complete, CompleteOptions
from snowflake.snowpark.functions import approx_count_distinct, col
from snowflake.snowpark.window import Window

USER_PROMPT_TEMPLATE_DESCRIBE_DATA = """
You are provided with a dataframe that has the following statistics for every column of my dataframe:
* count (total count of values)
* max (max value for numerical columns, first alphabetical value for categorical columns)
* mean (mean value for numerical columns, not applicable for categorical columns)
* min (min value for numerical columns, last alphabetical value for categorical columns)
* stddev (standard deviation for numerical columns, not applicable for categorical columns)
* top (most common value for categorical variables)
* freq (most common value’s frequency)
* unique (number of unique values for categorical columns, not applicable for numerical columns)
* datatype (string which is a categorical column, other values are numerical columns)

Based on this information, provide insights into potential data quality issues that could impact training of a machine learning model.  

Specifically, please identify and discuss:  

1. **Missing Values:** Columns with significantly lower `count` values compared to others, which could indicate missing data.  
2. **Outliers:** Columns where `min` or `max` values are far from the `mean`, or where `stddev` is unusually high, suggesting extreme values.  
3. **Data Skewness:** Columns where the `mean` is significantly different from the `min` and `max`, indicating skewed distributions.  
4. **Potential Data Type Issues:** Columns where numerical statistics may indicate categorical or incorrectly formatted data.  
5. **Feature Scaling Issues:** Columns with very large or very small values that might require normalization or standardization for ML models.  
6. **Other Anomalies:** Any unusual patterns that could suggest data corruption, incorrect data entry, or inconsistencies.  

Provide actionable recommendations on how to address any detected issues to improve the dataset's quality for machine learning.  
Make sure to base your recommendations on the type of model that will be trained which will be:
Model: {model_type}

Return these recommendations as a markdown table for streamlit's st.markdown() function with the following columns:
* Column Name
* Issue found (description of the actual issue found)
* Recommendation (description of steps recommended to mitigate the found issue)

Make sure there is one row per combination of column and issue.
Only return the markdown table.

Here is the output of `df.describe()`:  

{dataframe_sample}
"""

class CortexPilot():
    def __init__(self, session=None, llm='mistral-large2', temperature=0, top_p=0):
        if session is None:
            self.session = get_active_session()
        else:
            self.session = session
        self.llm = llm
        self.llm_options = CompleteOptions(
            temperature=temperature,
            top_p=top_p
        )
        self.dataframe_type_icons = {
            "pandas.core.frame.DataFrame": "🐼",
            "snowflake.snowpark.dataframe.DataFrame": "❄️",
            "snowflake.snowpark.table.Table": "❄️"
        }
        st.session_state['suggested_prompts'] = None
        
    def f_cortex_helper_describe_data(self, df, model_type='XGBoost Classifier'):
        """
        Requests the LLM to analyze potential issues in your data when used as training data for a machine learning model, based on the output of Snowpark's `describe()` function.  
        Returns the analyzed dataframe along with a summary of detected issues and recommendations for improvement given a model type.
        """
        if isinstance(df, pd.DataFrame):
            df = self.session.create_dataframe(df)
        df_describe = self._get_dataframe_description(df).reset_index(drop=True)
        #df_describe = df.describe().to_pandas()
        df_for_prompt = df_describe.to_markdown()
        user_query = USER_PROMPT_TEMPLATE_DESCRIBE_DATA.format(dataframe_sample = df_for_prompt, model_type=model_type)
        llm_input = [{"role": "user", "content": user_query}]
        llm_response = complete(model=self.llm, prompt=llm_input, options=self.llm_options)
        return df_describe, llm_response

    def _analyze_unique_values(self, df):
        """
        Get unique counts per string column.
        """
        categorical_columns = [col[0] for col in df.dtypes if col[1].startswith("string")]
        df_unique_counts = df.select(
            [approx_count_distinct(col(c)).alias(f"{c}") for c in categorical_columns]
        ).to_pandas()
        
        df_unique_counts['SUMMARY'] = 'unique'
        return df_unique_counts
    
    def _analyze_column_datatypes(self, df, describe_columns):
        """
        Create a dataframe with column names and their datatype
        """
        df_dtypes = pd.DataFrame(df.select(describe_columns).dtypes)
        #df_dtypes[1] = df_dtypes[1].apply(lambda x: x.replace('(16777216)',''))
        df_dtypes[1] = df_dtypes[1].str.replace(r"\(.*\)", "", regex=True)
        df_dtypes = df_dtypes.set_index(0).T
        df_dtypes['SUMMARY'] = 'datatype'
        return df_dtypes
    
    def _describe(self, df):
        """
        Run describe on provided dataframe and return list of of columns
        """
        describe_result = df.describe().to_pandas().round(3)
        describe_columns = list(describe_result.drop('SUMMARY',axis=1).columns)
        return describe_result, describe_columns
    
    def _get_dataframe_description(self, df):
        """
        Describe dataframe using Snowpark describe() and enrich it with additional analysis.
        """
        describe_result, describe_columns = self._describe(df)
        df_unique_counts = self._analyze_unique_values(df)
        df_top_freq = self._get_freq_top_(df)
        df_dtypes = self._analyze_column_datatypes(df, describe_columns)
        df_describe = pd.concat([describe_result, df_top_freq, df_unique_counts, df_dtypes])
        df_describe = df_describe.where(pd.notna(df_describe), '')
        return df_describe

    def _get_freq_top_(self, df):
        categorical_columns = [col[0] for col in df.dtypes if col[1].startswith("string")]
        for col_i, _col in enumerate(categorical_columns):
            window_spec = Window.order_by(col("count").desc(), col(_col))
            _col_df = (
                df.group_by(_col)
                .agg(F.count("*").alias("count"))
                .with_column("rank", F.rank().over(window_spec))
                .filter(col("rank") == 1)
                .select(col(_col).alias('"top"'), col("count").alias('"freq"'))
            )
            if col_i == 0:
                top_freq_df = _col_df
            else:
                top_freq_df = top_freq_df.union_all_by_name(_col_df)
        top_freq_df = top_freq_df.to_pandas()
        top_freq_df.index = categorical_columns
        top_freq_df = top_freq_df.T
        top_freq_df = top_freq_df.reset_index(names='SUMMARY')
        return top_freq_df

In [ ]:
# Get an instance of CortexPilot
my_pilot = CortexPilot(llm='claude-3-5-sonnet')

In [ ]:
transactions_df2 = transactions_df.with_column('A', lit('abc'))

In [ ]:
df_describe, llm_response = my_pilot.f_cortex_helper_describe_data(transactions_df2.drop('CUSTOMER_ID'), model_type='linear regression')
st.dataframe(df_describe)
st.markdown(llm_response)

In [ ]:
from snowflake.snowpark import Session
#from snowflake.snowpark.functions import col, count, rank
from snowflake.snowpark.window import Window

def most_frequent_value(df, columns):

    for col_i, _col in enumerate(columns):
        print(_col)
        window_spec = Window.order_by(col("count").desc(), col(_col))
        _col_df = (
            df.group_by(_col)
            .agg(F.count("*").alias("count"))
            .with_column("rank", F.rank().over(window_spec))
            .filter(col("rank") == 1)
            .select(col(_col).alias('"top"'), col("count").alias('"freq"'))
        )
        _col_df.show()
        if col_i == 0:
            top_freq_df = _col_df
        else:
            top_freq_df = top_freq_df.union_all_by_name(_col_df)
    top_freq_df = top_freq_df.to_pandas()
    top_freq_df.index = columns
    return top_freq_df

In [ ]:
a = most_frequent_value(transactions_df, ['TRANSACTION_CHANNEL'])
a

In [ ]:
a.T.reset_index(names='SUMMARY')

In [ ]:
transactions_df.limit(20).to_pandas().describe(include='all')

In [ ]:
df_describe, llm_response = my_pilot.f_cortex_helper_describe_data(transactions_df.drop('CUSTOMER_ID'))
st.dataframe(df_describe)
st.markdown(llm_response)

In [ ]:
df = transactions_df

In [ ]:
features.show()

In [ ]:
df_describe, llm_response = my_pilot.f_cortex_helper_analyze_describe_output(transactions_df)
st.markdown(llm_response)

In [ ]:
# Open the UI
my_pilot.ui_plotting()

When we plot the distribution of ONLINE vs. IN_SHOP revenue, we can see that 75% of our revenue comes from customer transactions that go into our shops.  
A model trained on this data should recognize that IN_SHOP transactions are the major driver of future customer revenue.

In [ ]:
my_pilot.f_cortex_helper_visualize_query(transactions_df, 'What was the total transaction amount per channel? Use a pie chart.')

# 3 - Feature Store & Feature Engineering
The Snowflake Feature Store enables data scientists and ML engineers to create, manage, and utilize machine learning features within machine learning pipelines.  
A feature store consists of feature views, which encapsulate Python or SQL pipelines that transform raw data into one or more related features.  
All features within a feature view are refreshed simultaneously from the source data.

Feature store objects are implemented as Snowflake objects and all feature store objects are therefore subject to Snowflake access control rules.
| Feature Store Object    | Snowflake Object |
| -------- | ------- |
| `FeatureStore` | Schema     |
| `Entity`    | Tag    |
| `FeatureView`  | Dynamic Table or View    |
| `Feature`  | Column in a Dynamic Table or View    |

### Setup the Feature Store
We are creating (or referencing if it already exists) a Feature Store that is stored in the schema `FEATURE_STORE`.  
The `default_warehouse` will be used to refresh features automatically.

In [ ]:
fs = FeatureStore(
    session=session, 
    database='SIMPLE_MLOPS_DEMO', 
    name='FEATURE_STORE', 
    default_warehouse='FEATURE_STORE_WH',
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

### Create a Feature Store Entity
Feature views are organized in the feature store according to the entities to which they apply. An entity is a higher-level abstraction that represents the subject matter of a feature.  
In our example, the main entity is the `CUSTOMER` and the features we will create will be linked to this entity.

In [ ]:
# Create a new entity for the Feature Store
entity = Entity(name="CUSTOMER", join_keys=["CUSTOMER_ID"], desc='Unique identifier for customers.')
fs.register_entity(entity)
fs.list_entities().show()

### Develop Features for Customer Transactions

The Snowpark Python API provides analytics functions for easily defining many common feature types, such as windowed aggregations.  
We will use `analytics.time_series_agg()` to quickly generate revenue for the past 1, 2 and 3 months per customer per channel which we will use as features for our machine learning model.

The feature dataframe should have the following columns:
| Column    | Purpose |
| -------- | ------- |
| `CUSTOMER_ID` | Identify relevant rows for the calculated feature (Join-Criteria)     |
| `DATE`    | Allow correct Point-in-Time Joins   |
| `Feature columns`  | Actual features per entity    |  

You can find more functions for quickly generating featueres here:  
[Common feature and query patterns](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/examples)

In [ ]:
def col_formatter(input_col, agg, window):
    feature_name = f"{agg.replace('SUM','TOTAL')}_{input_col}_{window.replace('-', 'past_').replace('MM','_MONTHS')}"
    return feature_name

in_shop_transaction_features = (
    transactions_df.filter(col('TRANSACTION_CHANNEL') == 'IN_SHOP')
    .group_by(['CUSTOMER_ID','DATE']).agg(F.sum('TRANSACTION_AMOUNT').as_('REVENUE'))
    .rename({'REVENUE':'REVENUE_IN_SHOP'})
    .analytics.time_series_agg(
        aggs={'REVENUE_IN_SHOP':['SUM']},
        windows=['-1MM','-2MM','-3MM'],
        sliding_interval="1D",
        group_by=['CUSTOMER_ID'],
        time_col='DATE',
        col_formatter=col_formatter
    ).drop(['SLIDING_POINT','REVENUE_IN_SHOP'])
)

online_transaction_features = (
    transactions_df.filter(col('TRANSACTION_CHANNEL') == 'ONLINE')
    .group_by(['CUSTOMER_ID','DATE']).agg(F.sum('TRANSACTION_AMOUNT').as_('REVENUE'))
    .rename({'REVENUE':'REVENUE_ONLINE'})
    .analytics.time_series_agg(
        aggs={'REVENUE_ONLINE':['SUM']},
        windows=['-1MM','-2MM','-3MM'],
        sliding_interval="1D",
        group_by=['CUSTOMER_ID'],
        time_col='DATE',
        col_formatter=col_formatter
    ).drop(['SLIDING_POINT','REVENUE_ONLINE'])
)

In [ ]:
online_transaction_features.show()

**Feature Descriptions**  
To avoid manually writing descriptions, we can use `complete()` to have an LLM generate JSON files containing business descriptions.  
The CortexPilot also offers a convenient function `f_describe_columns()` based on the complete() function.  
These descriptions are stored in the Feature Store alongside our features.

In [ ]:
feature_descriptions_in_shop_transactions = my_pilot.f_describe_columns(in_shop_transaction_features, exclude_columns=['CUSTOMER_ID','DATE'])
feature_descriptions_online_transactions = my_pilot.f_describe_columns(online_transaction_features, exclude_columns=['CUSTOMER_ID','DATE'])

st.json(feature_descriptions_in_shop_transactions)
st.json(feature_descriptions_online_transactions)

### Registering Feature Views
The `FeatureView` class accepts a Snowpark DataFrame object that contains the feature transformation logic. This allows you to define your features using any method supported by the Snowpark DataFrame API or Snowflake SQL. You can pass the DataFrame directly to the `FeatureView` constructor.  

Each `FeatureView` is associated with the corresponding `Entity`.  
The `refresh_freq` parameter determines how often the Feature Store checks for new data and updates the features automatically. For demonstration purposes, this value is set to 1 minute, but it should be adjusted based on the specific use case.

In [ ]:
# Create Feature View
in_shop_transaction_fv = FeatureView(
    name="IN_SHOP_REVENUE_FEATURES", 
    entities=[entity],
    timestamp_col='DATE',
    feature_df=in_shop_transaction_features, 
    refresh_freq="1 minute",
    refresh_mode='AUTO',
    desc="Features for in-shop transactions",
    overwrite=True
)

# Add descriptions for features
in_shop_transaction_fv = in_shop_transaction_fv.attach_feature_desc(feature_descriptions_in_shop_transactions)

in_shop_transaction_fv = fs.register_feature_view(
    feature_view=in_shop_transaction_fv, 
    version="V1", 
    block=True,
    overwrite=True
)

# Create Feature View
online_transaction_fv = FeatureView(
    name="ONLINE_REVENUE_FEATURES", 
    entities=[entity],
    timestamp_col='DATE',
    feature_df=online_transaction_features, 
    refresh_freq="1 minute",
    refresh_mode='AUTO',
    desc="Features for online transactions",
    overwrite=True
)

# Add descriptions for features
online_transaction_fv = online_transaction_fv.attach_feature_desc(feature_descriptions_online_transactions)

online_transaction_fv = fs.register_feature_view(
    feature_view=online_transaction_fv, 
    version="V1", 
    block=True,
    overwrite=True
)

### Discovering Features via Feature Store UI
After creating entities and feature views, you can utilize the [Feature Store User Interface](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/feature-store-ui) in Snowsight to locate the objects you need.  

Example of the Feature Store UI:  
![text](https://github.com/michaelgorkow/snowflake_simple_mlops/blob/main/resources/feature_store.png?raw=true)

In [ ]:
get_snowsight_url(session, 'Link to Feature Store', '#/features/database/SIMPLE_MLOPS_DEMO/store/FEATURE_STORE/entities')

### Discovering Features via Feature Store API
If you don't want to use the UI or want to develop workflows based on data in the feature store you can use the [Feature Store APIs](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/feature_store).  

CortexPilot can also help you in understanding how certain columns in a SQL query are calculated. In this example we are asking how a certain feature in the feature store is calculated but it works for any SQL query.  
Simply call `f_explain_column_sql` with the column name and the SQL query.

In [ ]:
st.markdown('### List of all Feature Views:')
sdf(fs.list_feature_views())

# Retrieve a Feature View
retrieved_feature_view = fs.get_feature_view(name='IN_SHOP_REVENUE_FEATURES',version='V1')

st.markdown('### Feature View Columns:')
sdf(retrieved_feature_view.list_columns())#.show(max_width=200)

# Manually refresh a Feature View
fs.refresh_feature_view(retrieved_feature_view)

st.markdown('### Feature View Refresh History:')
sdf(fs.get_refresh_history(retrieved_feature_view).order_by(col('REFRESH_END_TIME').desc()).limit(3))

# Explore lineage information
st.markdown('### Feature View Lineage:')
st.json(retrieved_feature_view.lineage(direction='both'))

# Use an LLM and the underlying SQL query to explain how the feature is calculated
st.markdown('### LLM Explanation for a Feature in the Feature Store:')
sql_explanation = my_pilot.f_explain_column_sql(column='TOTAL_REVENUE_IN_SHOP_PAST_1_MONTHS', sql_query=retrieved_feature_view.query)
st.markdown(sql_explanation)

# 4 - Model Training

### Generate the Training Dataset with Features from Feature Store
Our goal is to predict each customer's revenue for the next month based on their transactions from the past three months.  

We have data from January to April 2024. To define our target variable, `NEXT_MONTH_REVENUE`, we sum all transactions from April for each customer. To ensure proper point-in-time feature retrieval and avoid using future data, we only include transaction features up to **April 1st, 2024**, and mark this cutoff with the `FEATURE_CUTOFF_DATE` column.  

The DataFrame you just created is a **spine DataFrame**, which acts as a reference table linking customers (`CUSTOMER_ID`) with a timestamp (`FEATURE_CUTOFF_DATE`). It ensures consistent and reproducible feature retrieval in a **feature store**.  

Using this spine, you can generate a training dataset with [`generate_dataset()`](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/modeling#generating-snowflake-datasets-for-training). The Feature Store will automatically retrieve features as they were valid on that date and add them to the dataset.  

A [Snowflake Dataset](https://docs.snowflake.com/en/developer-guide/snowflake-ml/dataset) is a schema-level object designed for machine learning. It stores data in versions, ensuring immutability, efficient access, and compatibility with ML frameworks.

In [ ]:
target_df = session.table('SIMPLE_MLOPS_DEMO.RETAIL_DATA.TRANSACTIONS')
target_df = (
    target_df.filter(col('DATE').between('2024-04-02','2024-05-01'))    # Generate Target Variable for April 2024
    .group_by('CUSTOMER_ID')
    .agg(F.sum('TRANSACTION_AMOUNT').as_('NEXT_MONTH_REVENUE'))
    .with_column('FEATURE_CUTOFF_DATE', F.to_date(lit('2024-04-01')))   # Features until End of March 2024
)

# Get list of all customers
customers_df = session.table('SIMPLE_MLOPS_DEMO.RETAIL_DATA.CUSTOMERS').select('CUSTOMER_ID').distinct()

# Create spine dataframe
spine_df = target_df.join(customers_df, on=['CUSTOMER_ID'], how='outer')
spine_df = spine_df.fillna(0, subset='NEXT_MONTH_REVENUE')
spine_df.order_by('CUSTOMER_ID').show()

In [ ]:
train_dataset = fs.generate_dataset(
    name="SIMPLE_MLOPS_DEMO.FEATURE_STORE.NEXT_MONTH_REVENUE_DATASET",
    spine_df=spine_df,
    features=[in_shop_transaction_fv, online_transaction_fv],
    version="V1",
    spine_timestamp_col="FEATURE_CUTOFF_DATE",
    spine_label_cols=["NEXT_MONTH_REVENUE"],
    include_feature_view_timestamp_col=False,
    desc="Initial Training Dataset"
)

df = train_dataset.read.to_snowpark_dataframe()
df.show()

### Train an XGBoost Model
We randomly split the data, allocating **90% for training** and **10% for validation**.  
The training data is then used to train an **XGBoost regression model** with the `XGBRegressor` from the **Snowflake ML library**.

In [ ]:
# Split the data into train and test sets
train_df, test_df = df.random_split(weights=[0.9, 0.1], seed=0)

print(f'Number of samples in train: {train_df.count()}')
print(f'Number of samples in test: {test_df.count()}')

feature_columns = train_df.drop(['CUSTOMER_ID','FEATURE_CUTOFF_DATE','NEXT_MONTH_REVENUE']).columns

xgb_model = XGBRegressor(
    input_cols=feature_columns,
    label_cols=['NEXT_MONTH_REVENUE'],
    output_cols=['NEXT_MONTH_REVENUE_PREDICTION'],
    n_estimators=100,
    learning_rate=0.05,
    random_state=0
)

xgb_model = xgb_model.fit(train_df)

### Evaluate the XGBoost Model
You can immediately use the model’s `predict()` function to generate predictions on the test data.  
Snowflake ML also provides built-in metric functions, such as **Mean Absolute Percentage Error (MAPE)**, for evaluating model performance.  

Additionally, you can convert the model back to its native open-source format using `xgb_model.to_xgboost()`.  
This allows you to access feature importance values, which we visualize to better understand what influences the model’s predictions.  

As shown in the plot, the model correctly identified that **IN_SHOP transactions** are the primary driver of the target variable, `NEXT_MONTH_REVENUE`.

In [ ]:
predictions = xgb_model.predict(test_df)
# Analyze results
mape = mean_absolute_percentage_error(
    df=predictions, 
    y_true_col_names="NEXT_MONTH_REVENUE", 
    y_pred_col_names="NEXT_MONTH_REVENUE_PREDICTION"
)

st.info(f"**Mean absolute percentage error:** {mape:.5f}")

col1, col2 = st.columns(2)
with col1:
    # Plot Feature Importance
    plot_data = pd.DataFrame(
        list(zip(feature_columns, xgb_model.to_xgboost().feature_importances_)), 
        columns=['FEATURE','IMPORTANCE']
    )
    
    fig = px.bar(
        plot_data.sort_values('IMPORTANCE', ascending=True).head(10),
        x="IMPORTANCE",
        y="FEATURE",
        title="Feature Importance",
        labels={"FEATURE": "Feature", "IMPORTANCE": "Importance"},
        orientation="h"
    )
    fig.update_layout(title_font=dict(size=20, family="Arial", color="black"))
    st.plotly_chart(fig, use_container_width=True)
with col2:
    # Plot Predictions
    fig = px.scatter(
        predictions["NEXT_MONTH_REVENUE", "NEXT_MONTH_REVENUE_PREDICTION"].to_pandas().astype("float64"),
        x="NEXT_MONTH_REVENUE",
        y="NEXT_MONTH_REVENUE_PREDICTION",
        title="Actual vs. Predicted Revenue",
        labels={
            "NEXT_MONTH_REVENUE": "Actual Revenue",
            "NEXT_MONTH_REVENUE_PREDICTION": "Predicted Revenue"
        },
        trendline="ols",
        trendline_color_override="red"
    )
    fig.update_layout(title_font=dict(size=20, family="Arial", color="black"))
    st.plotly_chart(fig, use_container_width=True)

# 5 - Snowflake Model Registry
### Setup Model Registry
After training a model, the first step in operationalizing it and running inference in Snowflake is to **log the model in the Snowflake Model Registry**.  

The **Model Registry** allows you to securely manage models and their metadata in Snowflake, regardless of their origin or type, while also simplifying inference.  
It stores machine learning models as **first-class schema-level objects** within Snowflake.  

By setting `enable_monitoring` to True, the **Model Registry** can also be used for model monitoring, which we will implement in the next step.


In [ ]:
# Create reference to model registry
reg = Registry(
    session=session, 
    database_name='SIMPLE_MLOPS_DEMO', 
    schema_name='MODEL_REGISTRY', 
    options={'enable_monitoring':True},
)

### Register Model in Model Registry
The Model Registry's `log_model()` function takes the model object and logs it to the registry.  
The **name** and **version** help ensure the correct model is retrieved for inference.  

Additionally, we log relevant metrics/information, including:  
- **MAPE (Mean Absolute Percentage Error)** calculated on the test dataset  
- **FEATURE_CUTOFF_DATE**   

We also specify the following parameters:  

| Variable               | Description  |
|------------------------|-------------|
| `sample_input_data`    | Sample input data used to infer model signatures, serve as background data for explanations, and capture data lineage. |
| `conda_dependencies`   | Specifies model dependencies, such as the XGBoost library. |
| `relax_version`        | Enforces specific dependency versions for compatibility and reproducibility. |
| `enable_explainability` | Adds an explainability function to the model, allowing us to better understand its predictions using SHAP values. |

In [ ]:
registered_model = reg.log_model(
    xgb_model,
    model_name="CUSTOMER_REVENUE_MODEL",
    version_name='V1',
    metrics={
        'MAPE':mape, 
        "TRAINING_DATA":{'FEATURE_CUTOFF_DATE':'2024-04-01'}
    },
    comment="Model trained using XGBoost to predict revenue per customer for next month.",
    conda_dependencies=['xgboost'],
    sample_input_data=train_df.select(feature_columns).limit(100),
    options={"relax_version": False, "enable_explainability": True}
)

### Operationalize Models
There are multiple ways to operationalize models using Snowflake's Model Registry.  
One simple approach is to use **aliases** for the model. By assigning the alias **`PRODUCTION`**, any inference pipeline referencing this alias will automatically use the correct production-ready model.  

When a new model version is trained and ready for deployment, you can seamlessly update production by **removing the alias from the current model** and **assigning it to the new model**.  
This method ensures that existing ML pipelines remain unchanged, reducing the need for manual updates while maintaining a smooth model deployment process.

In [ ]:
registered_model.set_alias('PRODUCTION')

In [ ]:
# Retrieve the production model in your pipelines like this
production_model = reg.get_model('CUSTOMER_REVENUE_MODEL').version('PRODUCTION')
production_model.show_metrics()

### Explore Models in the Model Registry UI
The [Model Registry UI]((https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/snowsight-ui)) in Snowsight enables you to discover and explore machine learning models available for use in Snowflake.  

To view a model's details, click on its corresponding row in the Models list.  
The details page provides essential information, including the model's description, tags, and versions.

Example of the Model Registry UI:  
![text](https://github.com/michaelgorkow/snowflake_simple_mlops/blob/main/resources/model_registry_ui.png?raw=true)

In [ ]:
get_snowsight_url(session, 'Link to Model Registry', '#/models')

### **Model Explainability**
Since we enabled `model_explainability` during model registration, we can now use the auto-generated `explain` function to compute SHAP values for each feature.

#### **What are SHAP (SHapley Additive exPlanations) values?**  
SHAP values provide a game-theoretic approach to interpreting machine learning predictions by fairly attributing contributions to each feature. They help explain both **global feature importance** and **individual predictions**, showing how each feature increases or decreases the model’s output. **Positive SHAP values** indicate features that push the prediction higher, while **negative values** lower it relative to the model’s baseline prediction.

Once computed, we can convert the SHAP values into a native [`shap.Explanation`](https://shap.readthedocs.io/en/latest/generated/shap.Explanation.html) object, which includes:

| Variable        | Description |
|----------------|-------------|
| `values`       | Contribution of each feature to the prediction (output from Snowflake’s `explain` function). |
| `base_values`  | Expected model output (typically the mean prediction). |
| `data`         | Feature values (used for visualization). |
| `feature_names` | Names of the features (optional but recommended). |

You can then leverage SHAP's built-in functions for visualization and deeper analysis.

In [ ]:
# Calculate Shap values predictions
explanations = production_model.run(predictions, function_name="explain")
explanations = explanations.rename({col:col.replace('"""', '').upper() for col in explanations.columns})
shap_columns = [col for col in explanations.columns if '_EXPLANATION' in col]
explanations = explanations.to_pandas()

# Create the native shap Explanation object
shap_exp = shap.Explanation(
    values = explanations[shap_columns].values,
    base_values = np.full((len(explanations),), explanations['NEXT_MONTH_REVENUE_PREDICTION'].mean()),
    data = explanations[feature_columns].values,
    feature_names=feature_columns
)

### Global Explainibility using SHAP

The **left plot** is a SHAP **summary plot**, which displays the impact of each feature on the model’s output. Each dot represents a single instance, with the color indicating the feature value (blue = low, pink = high). Features at the top are the most influential, and the x-axis shows whether they push predictions higher (positive SHAP values) or lower (negative SHAP values).  

The **right plot** is a SHAP **violin plot**, which shows the distribution of SHAP values for each feature. The width of the violin indicates the density of SHAP values, helping visualize how much a feature’s impact varies across different predictions. Both plots highlight that recent revenue metrics (e.g., "TOTAL_REVENUE_IN_SHOP_PAST_1_MONTH") strongly influence the model, with high revenue values generally increasing predictions.

In [ ]:
col1, col2 = st.columns(2)
with col1:
    shap.summary_plot(shap_exp)
with col2:
    shap.plots.violin(shap_exp)

### Local Explainibility using SHAP

On the left you see a **SHAP waterfall plot** which explains how a model arrived at a specific prediction by showing feature contributions. It starts with the **expected value** (baseline prediction) and adjusts it based on **SHAP values** of individual features. **Blue bars** represent features that **lowered** the prediction, while **red bars** indicate those that **increased** it. The final predicted value is obtained by sequentially adding these contributions to the baseline. This plot helps identify which features had the most impact and whether they pushed the prediction up or down.

On the right we are plotting the distribution of **online vs. in-shop transaction** for this specific customer.

In [ ]:
import matplotlib.pyplot as plt

selected_customer = st.selectbox('Select Customer:', explanations.sort_values(by='CUSTOMER_ID')['CUSTOMER_ID'].values)
index = explanations.index[explanations['CUSTOMER_ID'] == selected_customer][0]
col1, col2 = st.columns([0.6,0.4])
with col1:
    st.subheader('Shap Waterfall Plot')
    fig, ax = plt.subplots()
    ax.set_title("", fontsize=16)
    plt.sca(ax)
    shap.plots.waterfall(shap_exp[index], show=False)
    plt.close()
    st.pyplot(fig)
with col2:
    st.subheader('Customer Transaction ')
    demo_flow.get_customer_revenue_plot(transactions_df, int(index))

### Continious Model Monitoring
Model behavior can change over time due to factors such as **input drift, stale training assumptions, data pipeline issues, hardware and software updates**.

**ML Observability** enables you to monitor the quality of models registered in the **Snowflake Model Registry** across multiple dimensions, including **performance, drift, and volume**.  

To measure drift for model monitoring, we use two tables:  

| Table      | Description  |
|------------|-------------|
| `BASELINE` | Contains a snapshot of data similar to `SOURCE`. It is used as a reference for comparing future feature values and predictions. |
| `SOURCE`   | Stores future predictions and feature values for monitoring. |

In [ ]:
# Save baseline predictions
predictions = predictions.with_column('FEATURE_CUTOFF_DATE', F.col('FEATURE_CUTOFF_DATE').cast('timestamp'))
predictions.write.save_as_table('SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.MM_REVENUE_BASELINE_V1', mode='overwrite')
predictions.write.save_as_table('SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.MM_REVENUE_SOURCE_V1', mode='overwrite')

### Creating predictions for the next month
We now use the trained model on our **April data** to predict each customer's **revenue for May**.  
The workflow looks like this:  
1. Build a spine DataFrame
2. Retrieve Features from Feature Store 
3. Generate predictions

Now, let’s take this a step further with a **real-world challenge**:  
Imagine you didn’t train the model yourself. How would you determine which input features are required and where to source them?

The answer is simple:  
Query the **automatically captured lineage** information in Snowflake!  
The model is directly linked to the dataset used for training, which in turn is connected to the relevant Feature Views.

In [ ]:
feature_views = production_model.lineage(direction='upstream')[0].lineage(domain_filter=['feature_view'], direction='upstream')
[fv.name for fv in feature_views]

In [ ]:
def get_feature_df(model, feature_cutoff_date):
    # Use lineage information to retrieve the feature views of this model
    feature_views = model.lineage(direction='upstream')[0].lineage(domain_filter=['feature_view'], direction='upstream')
    
    # Create the spine dataframe containing all customers
    spine_df = (
        session.table('SIMPLE_MLOPS_DEMO.RETAIL_DATA.CUSTOMERS')
        .select('CUSTOMER_ID')
        .with_column('FEATURE_CUTOFF_DATE', F.to_date(lit(feature_cutoff_date)))
    )

    # Retrieve feature values from the Feature Store for the specified cutoff date.
    feature_df = fs.retrieve_feature_values(
        spine_df=spine_df,
        features=feature_views,
        spine_timestamp_col="FEATURE_CUTOFF_DATE"
    )
    return feature_df

In [ ]:
feature_df = get_feature_df(
    production_model, 
    feature_cutoff_date='2024-05-01', 
)

predictions = production_model.run(feature_df, function_name='PREDICT')
predictions.write.save_as_table(table_name='SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.MM_REVENUE_SOURCE_V1', mode='append', column_order='name')

# View predictions
print('Predictions [column=NEXT_MONTH_REVENUE_PREDICTION]:')
session.table('SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.MM_REVENUE_SOURCE_V1').show()

### Creating a Model Monitor  

We are setting up a **model monitor** to continuously calculate and track model performance and drift over time.  

These calculations are based on the **`BASELINE`** and **`SOURCE`** tables created earlier.  
Each model requires its own dedicated **model monitor** to ensure accurate tracking and evaluation.

In [ ]:
source_config = ModelMonitorSourceConfig(
    source='SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.MM_REVENUE_SOURCE_V1',
    timestamp_column='FEATURE_CUTOFF_DATE',
    id_columns=['CUSTOMER_ID'],
    prediction_score_columns=['NEXT_MONTH_REVENUE_PREDICTION'],
    actual_score_columns=['NEXT_MONTH_REVENUE'],
    baseline='SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.MM_REVENUE_BASELINE_V1'
)

monitor_config = ModelMonitorConfig(
    model_version=reg.get_model('CUSTOMER_REVENUE_MODEL').version('PRODUCTION'),
    model_function_name='predict',
    background_compute_warehouse_name='COMPUTE_WH',
    refresh_interval='1 minute',
    aggregation_window='1 day'
)

model_monitor = reg.add_monitor(
    name='SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.MM_V1',
    source_config=source_config,
    model_monitor_config=monitor_config
)

### Simulating the next month of Customer Transactions
Our model has predicted each customer's **revenue for May 2024** and stored the results in the **`SOURCE`** table.  
Next, we simulate the actual transactions for May and update the **true revenue values** for each customer in the **`SOURCE`** table.  
When the **model monitor** refreshes, it will use these updated values to calculate various **model performance metrics**, including the MAPE.

In [ ]:
# Add new transactions (created as part of the initial demo setup)
new_transactions = session.table('SIMPLE_MLOPS_DEMO._DATA_GENERATION._TRANSACTIONS').filter(col('DATE').between('2024-05-02','2024-06-01'))
new_transactions.write.save_as_table(table_name='SIMPLE_MLOPS_DEMO.RETAIL_DATA.TRANSACTIONS', mode='append')

# Calculate actual values
actual_values_df = (
    session.table('SIMPLE_MLOPS_DEMO.RETAIL_DATA.TRANSACTIONS')
    .filter(col('DATE').between('2024-05-02','2024-06-01'))
    .group_by(['CUSTOMER_ID'])
    .agg(F.sum('TRANSACTION_AMOUNT').as_('TOTAL_REVENUE'))
    .with_column('DATE', F.to_date(lit('2024-05-01')))
)

# Get list of all customers
customers_df = session.table('SIMPLE_MLOPS_DEMO.RETAIL_DATA.CUSTOMERS').select('CUSTOMER_ID').distinct()

# Assume 0 revenue for customers without transactions
actual_values_df = actual_values_df.join(customers_df, on=['CUSTOMER_ID'], how='outer')
actual_values_df = actual_values_df.fillna(0,subset='TOTAL_REVENUE')

# Update source table from model monitor
source_table = session.table('SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.MM_REVENUE_SOURCE_V1')
source_table.update(
    condition=(
        (source_table['FEATURE_CUTOFF_DATE'] == actual_values_df['DATE']) &
        (source_table['CUSTOMER_ID'] == actual_values_df['CUSTOMER_ID'])
    ),
    assignments={
        "NEXT_MONTH_REVENUE": actual_values_df['TOTAL_REVENUE'],
    },
    source=actual_values_df
)

## Simulate Customer Transactions until February 2025
For convenience, I encapsulated all the logic for simulating future months into the helper function `simulate_model_performance()`.  
We use this function to simulate the model's behavior until February 2025.

In [ ]:
start_date = '2024-06-01'
end_date = '2025-01-01'
demo_flow.simulate_model_performance(production_model, start_date, end_date, generate_data=True)

## Explore the Model Monitor
Navigate to the Model Monitor and observe the `MAPE` and `Jensen-Shannon Distance`  for the last months.  

You will notice the following:
* Declining Model Performance
    * :chart_with_upwards_trend: MAPE (Mean Average Percentage Error)
* Feature Drift
    * :chart_with_upwards_trend: Distance for TOTAL_REVENUE_IN_SHOP_PAST_1_MONTHS (less in shop transaction volume)
    * :chart_with_downwards_trend: Difference for TOTAL_REVENUE_ONLINE_PAST_1_MONTHS (more online transaction volume)

Why is that?  
Well, if we visualize the monthly revenue distribution, we can see that online revenue grew while in-shop transaction declined.

Instead of using the builtin UI, you can also query model monitor metrics using the following table functions and build your own visuals:
* [MODEL_MONITOR_PERFORMANCE_METRIC](https://docs.snowflake.com/en/sql-reference/functions/model-monitor-performance-metric)
* [MODEL_MONITOR_DRIFT_METRIC](https://docs.snowflake.com/en/sql-reference/functions/model-monitor-drift-metric)
* [MODEL_MONITOR_STAT_METRIC](https://docs.snowflake.com/en/sql-reference/functions/model-monitor-stat-metric)

In [ ]:
get_snowsight_url(session, 'Link to Model Monitor', '#/data/databases/SIMPLE_MLOPS_DEMO/schemas/MODEL_REGISTRY/model/CUSTOMER_REVENUE_MODEL/version/V1/monitors/MM_V1/dashboard')

### Query the model monitor
With the built-in functions, querying metrics from the model monitor becomes effortless, allowing for further analysis and visualization.
Additionally, this notebook includes a small helper for the model registry, enabling you to quickly navigate through your registered models, visualize their metrics, and compare multiple models in a single graph.

Moreover, the outputs from these model registry functions can be leveraged to create alerts and trigger automated tasks. Example use cases include:
* Sending an email notification to your ML engineer if a model's performance drops below a predefined threshold.
* Initiating an automated retraining of a model with fresh data using [Snowflake Tasks](https://docs.snowflake.com/en/user-guide/tasks-intro)

In [ ]:
session.table_function(
    "MODEL_MONITOR_DRIFT_METRIC",
    lit('MM_V1'),
    lit('WASSERSTEIN'),
    lit('TOTAL_REVENUE_IN_SHOP_PAST_1_MONTHS'),
    lit('1 day'),
    lit('2024-01-01'),
    lit('2024-12-31')
).show()

In [ ]:
model_registry_helper = ModelRegistryHelper(session, reg)

In [ ]:
model_registry_helper.plot_model_performance()

### Why is the new model performing better?
Let's ask CortexPilot what changed.

The bar chart displays the overall revenue per month, categorized by transaction channel (Online and In-Shop). It shows a notable increase in online transactions starting in June.

In [ ]:
my_pilot.f_cortex_helper_visualize_query(
    transactions_df, 
    'What was the overall revenue per channel and month? Use a stacked bar plot and use YY-Monthname for the x-axis. Make sure the x-axis is ordered by month.'
)

## Train a new Model Version  

Since **user behavior has changed**, we will train a **new version of our model** using fresh data.  

To streamline this process, I have encapsulated the entire training workflow into the helper function `train_new_model()`, which automates the following steps:  

- **Creates the spine DataFrame**, including the target variable.  
- **Retrieves features** from the Feature Store.  
- **Creates a Snowflake Dataset** from the training data (ensuring reproducibility with a snapshot).  
- **Trains a new XGBoost model**.  
- **Registers the model** in the Snowflake Model Registry.  
- **Sets up a new model monitor** to track performance and drift.  
- **Compares model performance** against the existing production model.  
- **Deploys the new model** if it outperforms the current one by assigning it the **"PRODUCTION"** alias.  

Since the training data includes **June, July, and August 2024** (covering training data up to **Septemer 1st, 2024**, and looking back three months), the model should recognize that **ONLINE transactions** have become a major driver of customer revenue.

In [ ]:
@sproc(
    name='SIMPLE_MLOPS_DEMO.PUBLIC.TRAIN_CUSTOMER_REVENUE_MODEL',
    is_permanent=True,
    replace=True,
    stage_location='SIMPLE_MLOPS_DEMO.PUBLIC.PIPELINES',
    packages=['snowflake-snowpark-python','snowflake-ml-python==1.7.4','snowflake-telemetry-python'],
    imports=['@SIMPLE_MLOPS_DEMO.PUBLIC.GITHUB_REPOSITORY_SNOWFLAKE_SIMPLE_MLOPS/branches/main/src/demo_extras/model_trainer.py'],
    execute_as='caller'
)
def train_new_model(session: Session, feature_views: dict, feature_cutoff_date: str, target_start_date: str, target_end_date: str, model_version: str) -> float:
    from model_trainer import ModelTrainer
    logger = logging.getLogger("logger.ModelTrainer")
    tracer = trace.get_tracer("tracer.ModelTrainer")
    
    logger.info('Starting model training.')
    with tracer.start_as_current_span("XGBoost Pipeline"):
        model_trainer = ModelTrainer(session)
        mape = model_trainer.train_new_model(feature_views, feature_cutoff_date, target_start_date, target_end_date, model_version)
    logger.info('Model training finished.')
    
    return mape

In [ ]:
feature_views = {'IN_SHOP_REVENUE_FEATURES':'V1', 'ONLINE_REVENUE_FEATURES':'V1'}
feature_cutoff_date = '2024-09-01'
target_start_date = '2024-09-02'
target_end_date = '2024-10-01'
model_version = 'V2'

mape_value = train_new_model(session, feature_views, feature_cutoff_date, target_start_date, target_end_date, model_version)
st.info(mape_value)

In [ ]:
from datetime import timedelta
from snowflake.core.task import StoredProcedureCall
from snowflake.core.task.dagv1 import DAG, DAGTask, DAGOperation
from snowflake.core import Root

root = Root(session)

with DAG('ML_TRAINING_PIPELINE', schedule=timedelta(minutes=3)) as dag:
    task_train = DAGTask(
        'ML_TRAINING',
        StoredProcedureCall(train_new_model, args=[feature_views, feature_cutoff_date, target_start_date, target_end_date, '']),
        warehouse='COMPUTE_WH'
    )

schema = root.databases["SIMPLE_MLOPS_DEMO"].schemas["PUBLIC"]
dag_op = DAGOperation(schema)
dag_op.deploy(dag, mode=CreateMode.or_replace)
dag_op.run(dag)

# Snowflake Logging & Tracing
Snowflake is capturing logs, metrics and traces for your Stored Procedures, UDFs, etc. automatically in [Event Tables](https://docs.snowflake.com/en/developer-guide/logging-tracing/event-table-setting-up).  

**Note:**  
You need to [enable telemetry collection](https://docs.snowflake.com/en/developer-guide/logging-tracing/logging-tracing-enabling). This was done during the demo setup for you.

In [ ]:
get_snowsight_url(session, 'Task History', '#/compute/history/tasks')

### Simulate Model performance for Model Version V2 until 2025-01-31
Once again, we are simulating **model performance** based on customer transactions up to **February 2025**.  
Make sure to check the **model monitor** to evaluate whether the new model version trained on more recent data performs better.  
Additionally, analyze the **feature drift**, where you’ll notice that the trend for the **V2 model** is much more stable.

In [ ]:
model = reg.get_model('CUSTOMER_REVENUE_MODEL').version('V2')
start_date = '2024-10-01'
end_date = '2025-01-01'
demo_flow.simulate_model_performance(model, start_date, end_date)

In [ ]:
model_registry_helper.plot_model_performance(update_data=True)

### Comparing the two Model Versions
We have already observed that the new model provides **significantly better predictions** for future customer revenue. However, we want to gain deeper insights into **why** this improvement occurred.  

To analyze this, we are generating the SHAP Summary Plot for both models for the latest data. This reveals that the new model recognizes a **much stronger influence** of past **ONLINE transactions** on future customer revenue.  

In [ ]:
model_registry_helper.update_registry_data()

shap_exp1 = model_registry_helper.get_model_explanations(
    reg.get_model('CUSTOMER_REVENUE_MODEL').version('V1'), feature_columns=feature_columns, feature_cutoff_date='2025-01-01'
)

shap_exp2 = model_registry_helper.get_model_explanations(
    reg.get_model('CUSTOMER_REVENUE_MODEL').version('V2'), feature_columns=feature_columns, feature_cutoff_date='2025-01-01'
)

col1, col2 = st.columns(2)
with col1:
    shap.summary_plot(shap_exp1)
with col2:
    shap.summary_plot(shap_exp2)

## ML Lineage
Even though you may not have noticed, you’ve been capturing **lineage information** throughout the development of your machine learning pipeline.  

You can retrieve this information using the built-in function `lineage.trace()` for further analysis.  
For example, you can use this data to **visualize the lineage directly in the notebook**.  

Additionally, Snowflake provides a **more user-friendly and interactive UI** that allows you to explore and monitor your machine learning pipeline:  
![text](https://github.com/michaelgorkow/snowflake_simple_mlops/blob/main/resources/ml_lineage3.png?raw=true)

As shown, the lineage captures a **comprehensive view** of your pipeline, tracking data transformations and dependencies from the **source tables**, through the **feature view**, the **training dataset**, and ultimately the **registered model** in the Model Registry.

In [ ]:
get_snowsight_url(session, 'Link to Lineage View', '#/data/databases/SIMPLE_MLOPS_DEMO/schemas/MODEL_REGISTRY/model/CUSTOMER_REVENUE_MODEL/version/V2/lineage')

In [ ]:
trace = session.lineage.trace(
    object_name='SIMPLE_MLOPS_DEMO.MODEL_REGISTRY.CUSTOMER_REVENUE_MODEL',
    object_version='V2',
    object_domain='model',
    direction='both',
    distance=2
)
trace.show()

In [ ]:
lineage_helper = LineageHelper()
lineage_helper.visualize_lineage(trace.to_pandas(), short_names=True)